In [2]:
import logging
import pandas as pd
from sklearn.linear_model import LinearRegression
import wandb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pickle
import os,sys,yaml
import requests

module_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))

if module_path not in sys.path:
    sys.path.append(module_path)
# Now you can import your module
from src.DataOps.feature_engg import feature_engg_class
from src.ModelOps.model_fit import ModelFit
from src.DataOps.feature_engg import feature_engg_class

In [3]:
#load NASA_API_KEY from .env file
import os
from dotenv import load_dotenv
load_dotenv()
ALPHA_VANTAGE_API_KEY=os.getenv("ALPHA_VANTAGE_API_KEY")

In [24]:
import requests
import json
from tqdm import tqdm


def request_stock_price_hist(symbol, token, sample = False):
    if sample == False:
        q_string = 'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={}&outputsize=full&apikey={}'
    else:
        q_string = 'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={}&apikey={}'

    print("Retrieving stock price data from Alpha Vantage (This may take a while)...")
    r = requests.get(q_string.format(symbol, token))
    print("Data has been successfully downloaded...")
    date = []
    colnames = list(range(0, 6))
    df = pd.DataFrame(columns = colnames)
    print("Sorting the retrieved data into a dataframe...")
    for i in tqdm(r.json()['Weekly Adjusted Time Series'].keys()):
        date.append(i)
        row = pd.DataFrame.from_dict(r.json()['Weekly Adjusted Time Series'][i], orient='index').reset_index().T[1:]
        df = pd.concat([df, row], ignore_index=True)
    df.columns = ["open", "high", "low", "close", "adjusted close", "volume", "dividend amount"]
    df['date'] = date
    return df


In [25]:
data=request_stock_price_hist('IBM', ALPHA_VANTAGE_API_KEY)

Retrieving stock price data from Alpha Vantage (This may take a while)...
Data has been successfully downloaded...
Sorting the retrieved data into a dataframe...


100%|██████████| 1278/1278 [00:01<00:00, 774.85it/s]


,open,high,low,close,adjusted close,volume,dividend amount,date
0,167.4000,168.2200,162.6200,165.7100,165.7100,22535596,0.0000,2024-05-03
1,182.4500,184.6800,165.6600,167.1300,167.1300,42329269,0.0000,2024-04-26
2,185.5700,187.4800,180.1700,181.5800,181.5800,16929550,0.0000,2024-04-19
3,189.2400,191.2500,181.6850,182.2700,182.2700,14955313,0.0000,2024-04-12
4,190.0000,193.2800,187.3400,189.1400,189.1400,12808073,0.0000,2024-04-05
...,...,...,...,...,...,...,...,...
1273,113.0000,122.1200,107.5600,109.0000,56.7668,58626000,0.0000,1999-12-10
1274,104.9400,112.8700,102.1200,111.8700,58.2615,37670000,0.0000,1999-12-03
1275,105.5000,109.8700,101.8100,105.0000,54.6837,37165600,0.0000,1999-11-26
1276,96.0000,105.1200,92.6200,103.9400,54.1316,61550800,0.0000,1999-11-19
